### Import Required Libraries and Set Up Environment Variables

In [ ]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json

In [ ]:
# Set environment variables from the .env in the local environment
load_dotenv("example.env")

nyt_api_key = os.getenv("NYT_API_KEY")
tmdb_api_key = os.getenv("TMDB_API_KEY")


### Access the New York Times API

In [ ]:
# Set the base URL
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"

# Filter for movie reviews with "love" in the headline
# section_name should be "Movies"
# type_of_material should be "Review"
filter_query = 'section_name:"Movies" AND type_of_material:"Review" AND headline:"love"'

# Use a sort filter, sort by newest
sort = "newest"

# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"

# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"

# Build URL 
query_url = (
    f"{url}api-key={nyt_api_key}&begin_date={begin_date}&end_date={end_date}"
    + f'&fq={filter_query}&sort={sort}&fl={field_list}'
)


In [ ]:
# Create an empty list to store the reviews
reviews_list = []

# loop through pages 0-19
for page in range(20):
    # create query with a page number
    # API results show 10 articles at a time
    query_url = (
    f"{url}&api-key={nyt_api_key}&begin_date={begin_date}&end_date={end_date}"
    + f'&fq={filter_query}&sort={sort}&fl={field_list}&page={page}')
    
    
    # Make a "GET" request and retrieve the JSON
    response = requests.get(query_url)
    reviews = response.json()
    
    # Add a twelve second interval between queries to stay within API query limits
    time.sleep(12)
    
    # Try and save the reviews to the reviews_list
    try: 
        for review in reviews["response"]["docs"]:      

        # loop through the reviews["response"]["docs"] and append each review to the list
            reviews_list.append(review)

        # Print the page that was just retrieved
        print(f"Retrieved page: {page}")
    except:
        # Print the page number that had no results then break from the loop
        print(f"{page} had no results")
        break


In [ ]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(reviews_list[:5], indent=4))


In [ ]:
# Convert reviews_list to a Pandas DataFrame using json_normalize()
df_reviews = pd.json_normalize(reviews_list)
df_reviews.head()

In [ ]:
# Extract the title from the "headline.main" column and
# save it to a new column "title"
# Title is between unicode characters \u2018 and \u2019. 
# End string should include " Review" to avoid cutting title early
df_reviews['title'] = df_reviews['headline.main'].apply(
    lambda st: st[st.find("\u2018")+1:st.find("\u2019 Review")]
)

In [ ]:
# Extract 'name' and 'value' from items in "keywords" column
def extract_keywords(keyword_list):
    extracted_keywords = ""
    for item in keyword_list:
        # Extract 'name' and 'value'
        keyword = f"{item['name']}: {item['value']};" 
        # Append the keyword item to the extracted_keywords list
        extracted_keywords += keyword
    return extracted_keywords

# Fix the "keywords" column by converting cells from a list to a string
df_reviews['keywords'] = df_reviews['keywords'].apply(extract_keywords)

In [ ]:
# Create a list from the "title" column using to_list()
# These titles will be used in the query for The Movie Database
titles = df_reviews['title'].to_list()
print(titles)

### Access The Movie Database API

In [ ]:
# Prepare The Movie Database query
base_url = "https://api.themoviedb.org/3/search/movie?&api_key="
url = url + tmdb_api_key


In [ ]:
# Create an empty list to store the results
tmdb_movie_list = []

# Create a request counter to sleep the requests after a multiple
# of 50 requests
request_counter = 1 
# Loop through the titles
for title in titles:
    # Check if we need to sleep before making a request
    if request_counter % 50 == 0:
        print()
    # Add 1 to the request counter
        print("Sleeping...")
        time.sleep(1)
    full_url = query_url + title

    
    # Perform a "GET" request for The Movie Database
    
    # Include a try clause to search for the full movie details.
    # Use the except clause to print out a statement if a movie
    # is not found.
    request_counter += 1
    try:
        search_response = requests.get(full_url)
        search_data = search_response.json()
        # Get movie id
        movie_id = search_data['results'][0]['id']
        # Make a request for a the full movie details
        movie_details_url = f"https://api.themoviedb.org/3/movie/{movie_id}?api_key={tmdb_api_key}"
        details_response = requests.get(movie_details_url)
        movie_details = details_response.json()
        # Execute "GET" request with url
    
        # Extract the genre names into a list
        genres = [genre['name'] for genre in movie_details['genres']]


        # Extract the spoken_languages' English name into a list
        spoken_languages = [language['english_name'] for language in movie_details['spoken_languages']]
        # Extract the production_countries' name into a list
        production_countries = [country['name'] for country in movie_details['production_countries']]
        # Add the relevant data to a dictionary and
        # append it to the tmdb_movies_list list
        movie_dict = {
            'title': title,
            'original_title': movie_details['original_title'],
            'budget': movie_details['budget'],
            'original_language': movie_details['original_language'],
            'homepage': movie_details['homepage'],
            'overview': movie_details['overview'],
            'popularity': movie_details['popularity'],
            'runtime': movie_details['runtime'],
            'revenue': movie_details['revenue'],
            'release_date': movie_details['release_date'],
            'vote_average': movie_details['vote_average'],
            'vote_count': movie_details['vote_count'],
            'genres': genres,
            'spoken_languages': spoken_languages,
            'production_countries': production_countries
        }
        
        
        # Print out the title that was found
        tmdb_movie_list.append(movie_dict)
        print(f"Successfully added details for: {title}")
    except Exception:
        print(f"Movie not found or API limit reached for: {title}")




In [ ]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(tmdb_movie_list[:5], indent=4))

In [ ]:
# Convert the results to a DataFrame
tmdb_df = pd.DataFrame(tmdb_movie_list)

### Merge and Clean the Data for Export

In [ ]:
# Merge the New York Times reviews and TMDB DataFrames on title
merged_df = pd.merge(df_reviews,tmdb_df, on = "title")

In [ ]:
# Remove list brackets and quotation marks on the columns containing lists
# Create a list of the columns that need fixing
columns_to_fix = ['genres', 'spoken_languages', 'production_countries']

# Create a list of characters to remove
characters_to_remove = ['[', ']', "'"]


# Loop through the list of columns to fix
for column in columns_to_fix:
    # Convert the column to type 'str'
    for char in characters_to_remove:
    # Loop through characters to remove
        merged_df[column] = merged_df[column].astype(str).str.replace(char, ' ')


# Display the fixed DataFrame
merged_df

In [ ]:
# Drop "byline.person" column
merged_df = merged_df.drop(columns=["byline.person"])


In [ ]:
# Delete duplicate rows and reset index
merged_df = merged_df.drop_duplicates().reset_index(drop=True)

In [ ]:
# Export data to CSV without the index
merged_df.to_csv('final_data.csv', index=False)